In [1]:
import os, sys
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['PYSPARK_PYTHON'] =  'python3.9'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3.9'
os.environ['HADOOP_USER_NAME']='ssenigov'

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [ ]:
conf = (SparkConf().setAppName('Bucketing_Part2').setMaster('yarn') 
    .set("spark.sql.autoBroadcastJoinThreshold", -1)  
    .set("spark.sql.warehouse.dir", 'hdfs:/users/ssenigov/spark_warehouse') 
    .set("sql.legacy.bucketedTableScan.outputOrdering", True))
spark = SparkSession.builder.config(conf=conf).enableHiveSupport().getOrCreate()
print("app_id".ljust(40), spark.sparkContext.applicationId)

In [3]:
from faker import Faker
from pyspark.sql import Row
import random

# create data with faker module and make two sample dataframes with 10000 and 5000 rows 
fake = Faker()
Faker.seed(0)

people1 = [ Row(fake.first_name(), fake.last_name(), random.randint(1, 7)) for _ in range(10000)]
people2 = [ Row(fake.first_name(), fake.last_name(), random.randint(1, 7)) for _ in range(5000)]

df1 = spark.sparkContext.parallelize(people1, 2).toDF(["first_name", "last_name", 'birth_wd'])
df2 = spark.sparkContext.parallelize(people2, 2).toDF(["first_name", "last_name", 'birth_wd'])

In [4]:
dfrp1 = df1.repartition('first_name')
dfrp2 = df2.repartition('first_name')

In [5]:
dfrp1.write.bucketBy(5, 'first_name').mode("overwrite")\
  .saveAsTable('bucketed_people_rp_1', path='hdfs:/users/ssenigov/spark_warehouse/bprp1') ,

dfrp2.write.bucketBy(5, 'first_name').mode("overwrite")\
  .saveAsTable('bucketed_people_rp_2', path='hdfs:/users/ssenigov/spark_warehouse/bprp2')

In [6]:
df_bucketed_people_1 = spark.table("bucketed_people_rp_1")
df_bucketed_people_2 = spark.table("bucketed_people_rp_2")

In [7]:
dfrp1.write.bucketBy(5, 'first_name').partitionBy('birth_wd').mode("overwrite")\
  .saveAsTable('bucketed_people_rp_1', path='hdfs:/users/ssenigov/spark_warehouse/bpprp1') ,

dfrp2.write.bucketBy(5, 'first_name').partitionBy('birth_wd').mode("overwrite")\
  .saveAsTable('bucketed_people_rp_2', path='hdfs:/users/ssenigov/spark_warehouse/bpprp2')

In [8]:
spark.stop()